In [153]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

house_votes = pd.read_csv('house-votes-84.data',
                           sep= ',', header= None,names=['ClassName','handicapped','water','adoption','physician','elsalvador','religious','satellite','nicaraguan','missile','immigration','synfuels','education','superfund','crime','dutyfree','export'])




print ("Dataset Lenght:: ", len(house_votes))
print ("Dataset Shape:: ", house_votes.shape)


Dataset Lenght::  435
Dataset Shape::  (435, 17)


In [154]:
#remplacer les lettres et supprimer les valeurs oubliées
d = {'n':0,'y':1}
house_votes = house_votes.replace(d)


for col in house_votes.columns:
    house_votes=house_votes[house_votes[col]!='?']

In [155]:
print ("Dataset Lenght:: ", len(house_votes))
print ("Dataset Shape:: ", house_votes.shape)
house_votes.head()
house_votes.info()
house_votes.describe()

Dataset Lenght::  232
Dataset Shape::  (232, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 232 entries, 5 to 431
Data columns (total 17 columns):
ClassName      232 non-null object
handicapped    232 non-null object
water          232 non-null object
adoption       232 non-null object
physician      232 non-null object
elsalvador     232 non-null object
religious      232 non-null object
satellite      232 non-null object
nicaraguan     232 non-null object
missile        232 non-null object
immigration    232 non-null object
synfuels       232 non-null object
education      232 non-null object
superfund      232 non-null object
crime          232 non-null object
dutyfree       232 non-null object
export         232 non-null object
dtypes: object(17)
memory usage: 32.6+ KB


,ClassName,handicapped,water,adoption,physician,elsalvador,religious,satellite,nicaraguan,missile,immigration,synfuels,education,superfund,crime,dutyfree,export
count,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,0,0,1,0,1,1,1,1,0,1,0,0,1,1,0,1
freq,124,136,125,123,119,128,149,124,119,119,128,152,124,127,149,146,189


In [156]:
#1ere expérience : calculer la précision et le temps d'éxécution avec la méthode REF pour la séléction d'attributs

#Dataset split
array = house_votes.values
X = array[:,1:16]
y = array[:,0]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3)

In [157]:
lr = LogisticRegression(solver='lbfgs')
rfe = RFE(lr,4)
import time
start = time.time() 
fit=rfe.fit(X_train,y_train)
end = time.time()
print("Execution time with Recursive feature elimination REF is: %f"%(float(end)- float(start)))
y_pred = fit.predict(X_test)
confusion_matrix(y_test,y_pred)


Execution time with Recursive feature elimination REF is: 0.228878


array([[37,  0],
       [ 2, 31]], dtype=int64)

In [158]:
print("Num Features: ",fit.n_features_)
print("Selected Features:",fit.support_)
print("Feature Ranking: ", fit.ranking_)
print ("Accuracy with Recursive feature elimination is ", accuracy_score(y_test,y_pred)*100)

Num Features:  4
Selected Features: [False False  True  True False False False False False False  True False
 False  True False]
Feature Ranking:  [11  8  1  1  5  6  9 10  4  3  1  2 12  1  7]
Accuracy with Recursive feature elimination is  97.14285714285714


In [174]:
#2ème expérience : calculer la précision avec la méthode chi-square test pour la séléction d'attributs
# feature extraction
fit2= SelectKBest(chi2, k=4).fit(X, y)
numpy.set_printoptions(precision=3)
print(fit2.scores_)

X_trans=fit2.transform(X)
X_train_trans, X_test_trans, y_train_trans, y_test_trans = train_test_split(X_trans,y, test_size=0.3)


start = time.time() 
lr.fit(X_train_trans,y_train_trans)
end = time.time()
print("Execution time with chi-square test is: %f"%(float(end)- float(start)))

y_pred = lr.predict(X_test_trans)
confusion_matrix(y_test_trans,y_pred)



[1.970e+01 5.316e-02 5.296e+01 1.052e+02 5.918e+01 1.637e+01 2.674e+01
 5.242e+01 5.029e+01 1.829e-01 2.058e+01 6.479e+01 3.216e+01 3.621e+01
 3.673e+01]
Execution time with chi-square test is: 0.015992


array([[30,  2],
       [ 1, 37]], dtype=int64)

In [175]:
print ("Accuracy with chi-square test is ", accuracy_score(y_test_trans,y_pred)*100)

Accuracy with chi-square test is  95.71428571428572


In [180]:
#3ème expérience : calculer la précision et le temps d'éxécution avec la méthode decision tree pour la séléction d'attributs
et = ExtraTreesClassifier()
et.fit(X, y)
print(et.feature_importances_)

sfm = SelectFromModel(et, threshold=0.01)
fit3=sfm.fit(X,y)

X_trans=fit3.transform(X)
X_train_trans, X_test_trans, y_train_trans, y_test_trans = train_test_split(X_trans,y, test_size=0.3)

start = time.time() 
lr.fit(X_train_trans, y_train_trans)
end = time.time()
print("Execution time with decision tree is: %f"%(float(end)- float(start)))
print("Shape of the dataset ",shape)


pre = lr.predict(X_test_trans)
print ("Accuracy with tree based feature selection is ", accuracy_score(y_test_trans,pre)*100)

[0.011 0.02  0.038 0.367 0.128 0.007 0.036 0.062 0.016 0.013 0.053 0.096
 0.016 0.098 0.038]
Execution time with decision tree is: 0.015988
Shape of the dataset  (162, 11)
Accuracy with tree based feature selection is  97.14285714285714


C:\Users\user\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [187]:
#4eme expérience : calculer la précision et le temps d'éxécution sans la séléction d'attributs
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3)

start = time.time() 
fit=lr.fit(X_train,y_train)
end = time.time()
print("Execution time without any feature selection is: %f"%(float(end)- float(start)))
y_pred = fit.predict(X_test)
confusion_matrix(y_test,y_pred)

Execution time without any feature selection is: 0.033982


array([[36,  3],
       [ 0, 31]], dtype=int64)

In [188]:
print ("Accuracy without any feature selection is ", accuracy_score(y_test,y_pred)*100)

Accuracy without any feature selection is  95.71428571428572
